In [104]:
import numpy as np
from typing import List
import pandas as pd
import spacy
from gensim.models import CoherenceModel
from spacy.lang.en import English
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
import gensim
from gensim import corpora

PATH="dataset/"
FILE="sample_lyrics.csv"
PATHFILE=PATH+FILE

PATH="dataset/"
FILE="final_df.csv"
PATHFILE=PATH+FILE
FO_ID="final_df_id.csv"
FO="sample_df.csv"
STOP_WORDS = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
              "yourselves",
              "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them",
              "their",
              "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "iam",
              "youre", "is",
              "are",
              "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing",
              "a",
              "an",
              "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with",
              "about",
              "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from",
              "up",
              "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
              "there",
              "when",
              "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such",
              "no",
              "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just",
              "don",
              "should", "now", "oh", "\'", "?" ",", ":", ";", ",", ".", "!", "im", "oh","ima"]
#spacy_nlp = spacy.load('en_core_web_md')
SPACY_STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS


def set_corpus():
    corpus = []
    return corpus


def extract_words(text) -> List:
    """
    On vire les stopwords
    :param text: lyrics
    :return: tableau des mots des lyrics sans stop_word
    """
    corpus = text.split("\n")
    corpus_words = []
    words = []
    replacelist = set('( ) - \' ? : ; , . !'.split(' '))
    for line in corpus:
        for word in line.lower().split():
            if word not in SPACY_STOPWORDS:
                for char in word:
                    if char in replacelist:
                        word = word.replace(char, "")
                if word not in STOP_WORDS:
                    words.append(word)
        corpus_words.append(words)
        words = []
    #print("corpus words : \n",corpus_words)
    return corpus_words


def lemmatize_corpus(liste, threshold, exclusion=True):
    """
    :param list: corpus_words soir la liste des mots des lyrics
    :return: list de mots lemmatizée des lyrics
    """
    words = []
    corpus_words = liste
    word_frequency = {}
    lemmas_words = []
    for text in corpus_words:
        for token in text:
            if token in word_frequency:
                word_frequency[token] += 1
            else:
                word_frequency[token] = 1
    #print("word frequency : ",word_frequency)
    for text in corpus_words:
        for token in text:
            if exclusion:
                if word_frequency[token] > threshold:
                    words.append(token)
                    lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
                    lemmas = lemmatizer(token, u'NOUN')
                    lemmas_words.append(" ".join(lemmas))
            else:
                lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
                lemmas = lemmatizer(token, u'NOUN')
                # print("token : ",token," Lemmatization : ",lemmas)
                lemmas_words.append(" ".join(lemmas))
    #print("lemmas words : \n", lemmas_words)
    return lemmas_words


def add_to_corpus(corpus, liste) -> List:
    """
    on ajoute un sac de mots lemmatizés au corpus de texte
    :param corpus: corpus de lyrics
    :param text: chanson après lemmatization cleaning etc.
    :return:
    """
    return corpus.append(liste)


def lda_on_corpus(corpus):
    """
    lda sur le corpus de texte
    :param corpus: liste contenant les listes de mots des lyrics
    :return: gensim.models.ldamodel.LdaModel
    """
    id2word = corpora.Dictionary(corpus) # vectorisation de chaque mot du corpus
    texts = corpus
    corpus = [id2word.doc2bow(text) for text in texts] #mapping de (word_id,word_frequency)
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=1,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=5,
                                                alpha='auto',
                                                per_word_topics=True)
    #print(lda_model.print_topics())
    return lda_model


def extract_lda_values(lda_model):
    """
    retourne les values sorties par le modèle
    :param lda_model:
    :return: dictionnaire des valeurs retournées par le modèle
    """
    keyword_list = lda_model.print_topics()
    dic_id = {}
    dic_values = {}
    for val in keyword_list:
        key = val[0]
        value = ""
        donnee = val[1].replace("'", '')
        donnee = donnee.replace('"', "")
        tab = donnee.strip().split("+")
        for val in tab:
            val = val.strip().split("*")
            dic_values[val[1]] = val[0]
        #print(value)
        dic_id[key] = dic_values
    #print(dic_id)
    return dic_id

In [2]:
df = pd.read_csv(PATH+FILE)
#df.to_csv(PATH+FO_ID,index=True)


In [3]:
df = pd.read_csv(PATH+FO_ID)
df

,Unnamed: 0,Unnamed: 0.1,song,year,artist,genre,lyrics
0,0,249,i-got-that,2007,eazy-e,Hip-Hop,(horns)...\n(chorus)\nTimbo- When you hit me o...
1,1,250,8-ball-remix,2007,eazy-e,Hip-Hop,"Verse 1:\nI don't drink brass monkey, like to ..."
2,2,251,extra-special-thankz,2007,eazy-e,Hip-Hop,"19 muthaphukkin 93,\nand I'm back in this bitc..."
3,3,252,boyz-in-da-hood,2007,eazy-e,Hip-Hop,"Hey yo man, remember that shit Eazy did a whil..."
4,4,253,automoblie,2007,eazy-e,Hip-Hop,"Yo, Dre, man, I take this bitch out to the mov..."
5,5,254,i-d-rather-fuck-you,2007,eazy-e,Hip-Hop,"Aah, this is one of them songs\nYou can kick b..."
6,6,255,boyz-in-the-hood-remix,2007,eazy-e,Hip-Hop,"Hey yo man, remember that shit Eazy did a whil..."
7,7,256,fuck-dre,2007,eazy-e,Hip-Hop,Artist: Master P ++++++++++\nAlbum: Ghetto Dop...
8,8,257,trust-no-bitch,2007,eazy-e,Hip-Hop,Yes Im too smart to get dicked...or so the cas...
9,9,258,boyz-n-the-hood-g-mix,2007,eazy-e,Hip-Hop,"Hey yo man, remember that shit Eazy did a whil..."


In [6]:
sample = df.sample(30)

In [14]:
df.head(10)["lyrics"][0]

TypeError: 'DataFrame' object is not callable

In [107]:
cols = {
    "Unnamed: 0": [],
    "song": [],
    "year": [],
    "artist": [],
    "genre": [],
    "word0": [],
    "word1": [],
    "word2": [],
    "word3": [],
    "word4": [],
    "word5": [],
    "word6": [],
    "word7": [],
    "word8": [],
    "word9": []
}
empty_df = pd.DataFrame(cols)

for index, value in enumerate(sample["lyrics"]):
    corpus = set_corpus()
    corpus_words = extract_words(value)
    lemmas_words = lemmatize_corpus(corpus_words,1,exclusion=False)
    add_to_corpus(corpus,lemmas_words)
    lda_model = lda_on_corpus(corpus)
    var = extract_lda_values(lda_model)
    counter = 0
    for col in no_lyrics_df.columns.values:
        empty_df.loc[index, col] = no_lyrics_df.loc[no_lyrics_df.index[index]][col]
    for keyword, weight in var[0].items():
        empty_df.loc[index, "word"+str(counter)] = str((keyword, weight))
        counter += 1

empty_df  

,Unnamed: 0,song,year,artist,genre,word0,word1,word2,word3,word4,word5,word6,word7,word8,word9
0,108119.0,bad-for-each-other,2008.0,contraband,Rock,"('bad', '0.089')","('true', '0.056')","('sad', '0.056')","('good', '0.056')","('know', '0.048')","('feel', '0.040')","('why', '0.040')","('eye', '0.024')","('let', '0.024')","('sense', '0.024')"
1,226650.0,that-s-how-i-beat-shaq,2006.0,aaron-carter,Pop,"('jam', '0.033')","('like', '0.031')","('boom', '0.024')","('slam', '0.024')","('shaq', '0.022')","('heard', '0.020')","('cause', '0.018')","('beat', '0.015')","('aaron', '0.015')","('screaming', '0.015')"
2,286114.0,let-s-go-mano,2006.0,the-donnas,Rock,"('mano', '0.070')","('', '0.070')","('let', '0.063')","('go', '0.042')","('hit', '0.035')","('em', '0.035')","('theyre', '0.035')","('coast', '0.035')","('kid', '0.028')","('m', '0.028')"
3,27809.0,overture,2006.0,edguy,Metal,"('welcome', '0.151')","('run', '0.057')","('life', '0.057')","('damnation', '0.057')","('invitation', '0.057')","('fun', '0.057')","('party', '0.038')","('dance', '0.038')","('day', '0.038')","('waiting', '0.038')"
4,348250.0,thorns,2007.0,galadriel,Metal,"('heart', '0.041')","('like', '0.027')","('shadow', '0.027')","('beginning', '0.027')","('lie', '0.027')","('grief', '0.027')","('pain', '0.027')","('villainy', '0.027')","('tear', '0.027')","('inside', '0.027')"
5,328648.0,solar-prestige-a-gammon,1996.0,elton-john,Rock,"('prestige', '0.056')","('gammon', '0.044')","('solar', '0.044')","('pako', '0.033')","('paw', '0.022')","('kay', '0.022')","('kee', '0.022')","('zeta', '0.022')","('loto', '0.022')","('oo', '0.022')"
6,285955.0,be-quiet-drive,2006.0,deftones,Rock,"('away', '0.264')","('far', '0.218')","('dont', '0.173')","('care', '0.164')","('drive', '0.027')","('feel', '0.027')","('know', '0.018')","('fast', '0.018')","('said', '0.018')","('clothe', '0.018')"
7,342215.0,what-s-going-on,2006.0,the-dirty-dozen-brass-band,Jazz,"('what', '0.058')","('going', '0.058')","('right', '0.038')","('father', '0.032')","('picket', '0.032')","('baby', '0.026')","('way', '0.026')","('find', '0.026')","('brother', '0.026')","('got', '0.026')"
8,15122.0,surrender,2007.0,blake-lewis,Pop,"('surrender', '0.041')","('everybody', '0.041')","('something', '0.035')","('need', '0.035')","('away', '0.035')","('coming', '0.029')","('find', '0.023')","('reaching', '0.023')","('time', '0.023')","('heart', '0.023')"
9,322353.0,millworker,1979.0,bette-midler,Rock,"('rest', '0.036')","('life', '0.026')","('aint', '0.021')","('millwork', '0.021')","('mornin', '0.021')","('', '0.021')","('machine', '0.016')","('sailor', '0.016')","('afternoon', '0.016')","('man', '0.016')"


In [108]:
empty_df = empty_df.rename(columns={empty_df.columns.values[0]:"id"})

In [109]:
empty_df.to_csv(PATH+FO_ID,index=False)